# **Model Training**

Importing the Dependencies



In [1]:
import os
import numpy as np
import pandas as pd
import re
import random
import nltk
nltk.download("popular")
nltk.download('stopwords')
# Using the stopwords.
from nltk.corpus import stopwords
# Initialize the stopwords
stoplist = stopwords.words('english')
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib
import pickle
import tensorflow as tf
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
uploaded = files.upload()

Saving Context.csv to Context.csv


In [3]:
#Reading the file
nlptag = pd.read_csv('Context.csv')
nlptag.head(5)

,Text,Context/Topic
0,The eternal mystique of Goldman Sachs,Politics
1,Either you don't care enough to actually tell ...,Love
2,I am such an IDIOT.,Heavy Emotion
3,While lifting weights on Friday and doing bent...,Health
4,Something's watching me,Animals


In [4]:
#Renaming for the convenience
nlptag.rename(columns={"Context/Topic": "Topic"} , inplace = True)
nlptag['Topic']

0             Politics
1                 Love
2        Heavy Emotion
3               Health
4              Animals
             ...      
31381    Heavy Emotion
31382    Heavy Emotion
31383        Education
31384         Politics
31385         Politics
Name: Topic, Length: 31386, dtype: object

**Text cleaning and preprocessing**

In [5]:
##cleaning
nlptag['Cleaned'] = nlptag['Text'].apply(lambda x: "".join(x.lower() for x in str ((x.split()))))
nlptag['Cleaned'] = nlptag['Cleaned'].str.replace(r"[^a-zA-Z ]+"," ").replace('\s+', ' ',regex=True)
nlptag.head(5)
                                                                              

,Text,Topic,Cleaned
0,The eternal mystique of Goldman Sachs,Politics,the eternal mystique of goldman sachs
1,Either you don't care enough to actually tell ...,Love,either you don t care enough to actually tell...
2,I am such an IDIOT.,Heavy Emotion,i am such an idiot
3,While lifting weights on Friday and doing bent...,Health,while lifting weights on friday and doing ben...
4,Something's watching me,Animals,something s watching me


In [6]:
##Removing stopwords and lemmenting
stop = stopwords.words("english")
nlptag.Cleaned = nlptag.Cleaned.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
lemmatizer = WordNetLemmatizer()
nlptag.Cleaned=nlptag.Cleaned.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
nlptag['Cleaned_lemmatized']=nlptag.Cleaned.apply(lambda x:"".join([lemmatizer.lemmatize(y) for y in x]))
nlptag.head(5)

,Text,Topic,Cleaned,Cleaned_lemmatized
0,The eternal mystique of Goldman Sachs,Politics,eternal mystique goldman sachs,eternal mystique goldman sachs
1,Either you don't care enough to actually tell ...,Love,either care enough actually tell u feel dumbas...,either care enough actually tell u feel dumbas...
2,I am such an IDIOT.,Heavy Emotion,idiot,idiot
3,While lifting weights on Friday and doing bent...,Health,lifting weights friday bent rows felt sharp pa...,lifting weights friday bent rows felt sharp pa...
4,Something's watching me,Animals,something watching,something watching


In [7]:
#function for other cleaning
def cleansingLayer(line):
  try:
    line = line.lower().strip()
    line = re.sub(r'\s+', ' ',line)
    line = re.sub(r'thanks.*', '',line)
    line = re.sub(r"what's", "what is",line)
    line = re.sub(r"don't", "do not",line)
    line = re.sub(r"should've", "should have",line)
    line = re.sub(r"couldn't", "could not",line)
    line = re.sub(r"can't", "can not",line)
    line = re.sub(r"aren't", "are not",line)
    line = re.sub(r"didn't", "did not",line)
    line = re.sub(r"doesn't","does not",line)
    line = re.sub(r"hadn't", "had not",line)
    line = re.sub(r"hasn't","has not",line)
    line = re.sub(r"haven't","have not",line)
    line = re.sub(r"isn't", "is not",line)
    line = re.sub(r"mightn't","might not",line)
    line = re.sub(r"mustn't", "must not",line)
    line = re.sub(r"shouldn't", "should not",line)
    line = re.sub(r"wasn't", "was not",line)
    line = re.sub(r"weren't","were not",line)
    line = re.sub(r"wouldn't","would not",line)
    line = re.sub(r"\'ve", " have ",line)
    line = re.sub(r"n't", "not",line)
    line = re.sub(r"\'re", " are ",line)
    line = re.sub(r"\'d"," would",line)
    line = re.sub(r"\'ll"," will ",line)
    line = re.sub(r" abt "," about ",line)
    line = re.sub(r"n ot", " not ",line)
    lem = WordNetLemmatizer()
    line = ' '.join(lem.lemmatize(i, pos='v') for i in line.split())
    line = re.sub('[^a-zA-Z ]', ' ', line)
    line = re.sub(r'\s+', ' ',line)
    line = line.strip()

  except Exception as e:
     print(e)
     print(line)
     print('++++')

  return line   


In [8]:
nlptag = nlptag.drop_duplicates(['Cleaned','Topic'])
nlptag.shape

(30421, 4)

In [9]:
nlptag['Topic'].value_counts()

Heavy Emotion    3571
Religion         3363
Love             3106
Self             3028
Compliment       2784
Animals          2570
Health           2562
Education        2431
Joke             2429
Science          2395
Politics         2182
Name: Topic, dtype: int64

In [10]:
nlptag['Cleaned_lemmatized'] = nlptag['Cleaned_lemmatized'].apply(lambda x: cleansingLayer(str(x)))
nlptag['Cleaned_lemmatized'].replace('',np.nan, inplace=True)
nlptag.dropna(subset=['Cleaned_lemmatized'], inplace=True)

In [11]:
nlptag

,Text,Topic,Cleaned,Cleaned_lemmatized
0,The eternal mystique of Goldman Sachs,Politics,eternal mystique goldman sachs,eternal mystique goldman sachs
1,Either you don't care enough to actually tell ...,Love,either care enough actually tell u feel dumbas...,either care enough actually tell u feel dumbas...
2,I am such an IDIOT.,Heavy Emotion,idiot,idiot
3,While lifting weights on Friday and doing bent...,Health,lifting weights friday bent rows felt sharp pa...,lift weight friday bend row felt sharp pain lo...
4,Something's watching me,Animals,something watching,something watch
...,...,...,...,...
31380,4 types of jews as reported by an Israeli scholar,Religion,types jews reported israeli scholar,type jews report israeli scholar
31382,Looks are everything,Heavy Emotion,looks everything,look everything
31383,Teacher: write a thing that you want to do til...,Education,teacher write thing want till,teacher write thing want till
31384,"General Secretary of India's ruling party, the...",Politics,general secretary india ruling party bjp threa...,general secretary india rule party bjp threate...


In [12]:
X= nlptag['Cleaned_lemmatized']
y = nlptag['Topic']
x_train, x_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)

**Vectorizing the data using TFIDF**

In [13]:
tfid_text = TfidfVectorizer(ngram_range= (1,3),max_features=3149)
tfid_text

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=3149,
                min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [14]:
#Stroing the as the idf object
train_text_pkl = tfid_text.fit(x_train)
train_text = tfid_text.fit_transform(x_train)
test_text = tfid_text.transform(x_test)

In [15]:
#Saving it as a pickle to use for the feature tests
feature_name = tfid_text.get_feature_names()
joblib.dump(train_text_pkl,open("binary_object_stream","wb"))
with open('binary_object_stream','rb') as f:
  f_content = f.read()
tf.io.gfile.GFile('tfidf.pkl','wb').write(f_content)

In [16]:
f_content = tf.io.gfile.GFile('tfidf.pkl','rb').read()
with open('binary_object_stream','wb') as f:
  f.write(f_content)
tfidftest_sb = joblib.load(open("binary_object_stream","rb"))

tf1_new = TfidfVectorizer(ngram_range= (1,3),max_features=3149,vocabulary= tfidftest_sb.vocabulary_)
tdidftest = tf1_new.transform(x_test)

In [26]:
tdidftest = tf1_new.transform(x_test)

In [17]:
test_text.shape

(3032, 3149)

In [18]:
tdidftest.shape

(3032, 3149)

In [19]:
train_text.shape

(27280, 3149)

In [20]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=0)
#X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0)
model.fit(train_text, y_train)
y_pred = model.predict(test_text)
y_pred_proba = model.predict_proba(test_text)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [21]:
print("Accuracy score (training: {0:.3f}".format(model.score(train_text,y_train)))
print("Accuracy score(test): {0:.3f}".format(model.score(test_text,y_test)))

Accuracy score (training: 0.712
Accuracy score(test): 0.612


In [42]:
from sklearn.metrics import classification_report
logistic_pred=model.predict(test_text)
logistic_pred_train = model.predict(train_text)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(y_train,logistic_pred_train))
print(classification_report(y_test,logistic_pred))

               precision    recall  f1-score   support

      Animals       0.77      0.73      0.75      2341
   Compliment       0.78      0.70      0.74      2460
    Education       0.76      0.72      0.74      2181
       Health       0.72      0.70      0.71      2284
Heavy Emotion       0.63      0.68      0.66      3202
         Joke       0.76      0.75      0.76      2177
         Love       0.70      0.69      0.69      2788
     Politics       0.87      0.81      0.84      1967
     Religion       0.79      0.75      0.77      3028
      Science       0.69      0.65      0.67      2151
         Self       0.52      0.66      0.58      2701

     accuracy                           0.71     27280
    macro avg       0.73      0.71      0.72     27280
 weighted avg       0.72      0.71      0.71     27280

               precision    recall  f1-score   support

      Animals       0.63      0.64      0.63       226
   Compliment       0.72      0.66      0.69       269
    Ed

## Saving model to pkl

In [22]:
#Saving it as a pickle to use for the feature tests

joblib.dump(model,open("binary_object_stream","wb"))
with open('binary_object_stream','rb') as f:
  f_content = f.read()
tf.io.gfile.GFile('model.pkl','wb').write(f_content)

In [25]:
from google.colab import files
files.download('tfidf.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>